### Pipeline

The entire pipeline for comparing and see the correlation between the UST-HRV and the Standard one.

This process consist of several steps:
- Data Extraction 
- rPPG Validation (MAE /RMSE) for each Scenario with bland-altman aggrement 
- Making a section for each scenario (30 sec, 1min and standard one 3 min) for UBFC-Phys, (30 sec and 1 min) for Physio Itera
- Plot the correlation and check the limit within bland-altman
- Done.

### Kesimpulan

- Overall the Facial Landmark unggul / better than Face detector
- For rest task, it seems the `Face Detector` / `Facial Landmark` shows similar result with lower MAE / RMSE and low difference bias in Bland-Altman plot.
- Green is the worst one for both.
- Task 3 is the Face Detector having advantages, it's having slightly lower MAE / RMSE score compare to the Landmark, but still I don't consider it valid since the data maybe have some artifcats.

- For Physio-Itera
    - For some reason the Face Detector and Facial Landmark have similar result for in MAE and from the mean bias bland-altman shows higher mean bias.

- For future test, let's use the Base Task 1 and Task 2

## Data Extraction

This will structure the folder so it can be easier to access, all rPPG methods already be in the .npy files for easier acccess.

In [1]:
## Import Dependencies
import numpy as np
import mediapipe as mp
import pandas as pd
import cv2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import matplotlib.pyplot as plt
import scipy
import os
import neurokit2 as nk
from prettytable import PrettyTable

## UBFC-Phys Scenario-1 (Task 1)

Plot the data and see the data variation between models within tables views and compare with the GT

In [ ]:
root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44", "s45", "s46", "s47", "s48", "s49", "s50", "s51","s52","s53","s54","s55","s56"]
tasks = ["T1"]

table = PrettyTable()

## Create a table for the HRV Metrics for GT and RPPG
table.field_names = ["Subject", "PR_RPPG", "PR_GT", "MeanNN_RPPG", "MeanNN_GT", "SDNN_RPPG", "SDNN_GT", 
                     "RMSSD_RPPG", "RMSSD_GT", "pNN50_RPPG", "pNN50_GT", 
                     "LF_RPPG", "LF_GT", "HF_RPPG", "HF_GT", "LF_HF_Ratio_RPPG", 
                     "LF_HF_Ratio_GT"]

for subject in subjects:
    if not os.path.exists(f"{root_path}/{subject}"):
        print(f"Subject {subject} not found, skipping")
    
    for task in tasks:

        ## Create a dataFrame for storing the HRV Metrics
        rppg_hrv_metrics = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        gt_hrv_metrics = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }


        # Plot the path of the OMIT, POS, and LGI, CHROM and GREEN
        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))

        # Open the GT file
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        
        # Flatten the value for the GT
        GT = GT.flatten()

        # Process all the signals
        pos_hr = nk.ppg_process(pos, sampling_rate=35)[0]["PPG_Rate"].mean()
        gt_hr = nk.ppg_process(GT, sampling_rate=64)[0]["PPG_Rate"].mean()

        fs = 35
        signals, _ = nk.ppg_process(pos, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs)

        ## Compute the HRV Metrics
        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        rppg_hrv_metrics['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        rppg_hrv_metrics['SDNN'].append(hrv_time['HRV_SDNN'].item())
        rppg_hrv_metrics['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        rppg_hrv_metrics['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        rppg_hrv_metrics['LF'].append(hrv_freq['HRV_LF'].item())
        rppg_hrv_metrics['HF'].append(hrv_freq['HRV_HF'].item())
        rppg_hrv_metrics['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        rppg_hrv_metrics['PR'].append(signals['PPG_Rate'][0].item())

        fs = 64
        signals, _ = nk.ppg_process(GT, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs)

        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        gt_hrv_metrics['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        gt_hrv_metrics['SDNN'].append(hrv_time['HRV_SDNN'].item())
        gt_hrv_metrics['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        gt_hrv_metrics['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        gt_hrv_metrics['LF'].append(hrv_freq['HRV_LF'].item())
        gt_hrv_metrics['HF'].append(hrv_freq['HRV_HF'].item())
        gt_hrv_metrics['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        gt_hrv_metrics['PR'].append(signals["PPG_Rate"][0].item())

        ## Add the metrics to the table
        table.add_row([subject, 
                       rppg_hrv_metrics['PR'][0], gt_hrv_metrics['PR'][0],
                       rppg_hrv_metrics['MeanNN'][0], gt_hrv_metrics['MeanNN'][0],
                       rppg_hrv_metrics['SDNN'][0], gt_hrv_metrics['SDNN'][0],
                       rppg_hrv_metrics['RMSSD'][0], gt_hrv_metrics['RMSSD'][0],
                       rppg_hrv_metrics['pNN50'][0], gt_hrv_metrics['pNN50'][0],
                       rppg_hrv_metrics['LF'][0], gt_hrv_metrics['LF'][0],
                       rppg_hrv_metrics['HF'][0], gt_hrv_metrics['HF'][0],
                       rppg_hrv_metrics['LF_HF'][0], gt_hrv_metrics['LF_HF'][0]])

## Print the table
print(table)


+---------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+----------------------+----------------------+---------------------+-----------------------+---------------------+---------------------+
| Subject |      PR_RPPG      |       PR_GT        |    MeanNN_RPPG    |     MeanNN_GT      |     SDNN_RPPG      |      SDNN_GT       |     RMSSD_RPPG     |      RMSSD_GT      |     pNN50_RPPG    |      pNN50_GT      |       LF_RPPG        |        LF_GT         |       HF_RPPG       |         HF_GT         |   LF_HF_Ratio_RPPG  |    LF_HF_Ratio_GT   |
+---------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+----------------------+----------------------+---------------------+----

In [5]:
## Compute the MAE / RMSE for the RPPG and GT signals
root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44","s45","s46","s47","s48","s49","s50","s51","s52", "s53","s54","s55","s56",]
tasks = ["T1"]

table = PrettyTable()
## Create a table for the MAE / RMSE HRV metrics for GT and RPPG
table.field_names = ["Subject", "MAE_PR", "RMSE_PR", "MAE_MeanNN", "RMSE_MeanNN",
                     "MAE_SDNN", "RMSE_SDNN", "MAE_RMSSD", "RMSE_RMSSD", 
                     "MAE_pNN50", "RMSE_pNN50", "MAE_LF", "RMSE_LF", 
                     "MAE_HF", "RMSE_HF", "MAE_LF_HF_Ratio", "RMSE_LF_HF_Ratio"]

for subject in subjects:
    if not os.path.exists(f"{root_path}/{subject}"):
        print(f"Subject {subject} not found, skipping")
    
    for task in tasks:

        ## Create a dataFrame for storing the HRV Metrics
        rppg_hrv_metrics = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        gt_hrv_metrics = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))
        # Open the GT file
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        GT = GT.flatten()


        # Process all the signals
        pos_hr = nk.ppg_process(pos, sampling_rate=35)[0]["PPG_Rate"].mean()
        gt_hr = nk.ppg_process(GT, sampling_rate=64)[0]["PPG_Rate"].mean()

        fs = 35
        signals, _ = nk.ppg_process(pos, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs)

        ## Compute the HRV Metrics
        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        rppg_hrv_metrics['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        rppg_hrv_metrics['SDNN'].append(hrv_time['HRV_SDNN'].item())
        rppg_hrv_metrics['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        rppg_hrv_metrics['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        rppg_hrv_metrics['LF'].append(hrv_freq['HRV_LF'].item())
        rppg_hrv_metrics['HF'].append(hrv_freq['HRV_HF'].item())
        rppg_hrv_metrics['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        rppg_hrv_metrics['PR'].append(signals['PPG_Rate'][0].item())

        fs = 64
        signals, _ = nk.ppg_process(GT, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs)

        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        gt_hrv_metrics['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        gt_hrv_metrics['SDNN'].append(hrv_time['HRV_SDNN'].item())
        gt_hrv_metrics['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        gt_hrv_metrics['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        gt_hrv_metrics['LF'].append(hrv_freq['HRV_LF'].item())
        gt_hrv_metrics['HF'].append(hrv_freq['HRV_HF'].item())
        gt_hrv_metrics['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        gt_hrv_metrics['PR'].append(signals["PPG_Rate"][0].item())

        ## Compute the MAE / RMSE for the RPPG and GT signals
        rppg_pr = rppg_hrv_metrics['PR'][0]
        gt_pr = gt_hrv_metrics['PR'][0]
        rppg_meanNN = rppg_hrv_metrics['MeanNN'][0]
        gt_meanNN = gt_hrv_metrics['MeanNN'][0]
        rppg_sdnn = rppg_hrv_metrics['SDNN'][0]
        gt_sdnn = gt_hrv_metrics['SDNN'][0]
        rppg_rmssd = rppg_hrv_metrics['RMSSD'][0]
        gt_rmssd = gt_hrv_metrics['RMSSD'][0]
        rppg_pnn50 = rppg_hrv_metrics['pNN50'][0]
        gt_pnn50 = gt_hrv_metrics['pNN50'][0]
        rppg_lf = rppg_hrv_metrics['LF'][0]
        gt_lf = gt_hrv_metrics['LF'][0]
        rppg_hf = rppg_hrv_metrics['HF'][0]
        gt_hf = gt_hrv_metrics['HF'][0]
        rppg_lf_hf = rppg_hrv_metrics['LF_HF'][0]
        gt_lf_hf = gt_hrv_metrics['LF_HF'][0]

        # Compute the MAE
        mae_pr = np.abs(rppg_pr - gt_pr)
        mae_meanNN = np.abs(rppg_meanNN - gt_meanNN)
        mae_sdnn = np.abs(rppg_sdnn - gt_sdnn)
        mae_rmssd = np.abs(rppg_rmssd - gt_rmssd)
        mae_pnn50 = np.abs(rppg_pnn50 - gt_pnn50)
        mae_lf = np.abs(rppg_lf - gt_lf)
        mae_hf = np.abs(rppg_hf - gt_hf)
        mae_lf_hf = np.abs(rppg_lf_hf - gt_lf_hf)

        # Compute the RMSE
        rmse_pr = np.sqrt((rppg_pr - gt_pr) ** 2)
        rmse_meanNN = np.sqrt((rppg_meanNN - gt_meanNN) ** 2)
        rmse_sdnn = np.sqrt((rppg_sdnn - gt_sdnn) ** 2)
        rmse_rmssd = np.sqrt((rppg_rmssd - gt_rmssd) ** 2)
        rmse_pnn50 = np.sqrt((rppg_pnn50 - gt_pnn50) ** 2)
        rmse_lf = np.sqrt((rppg_lf - gt_lf) ** 2)
        rmse_hf = np.sqrt((rppg_hf - gt_hf) ** 2)
        rmse_lf_hf = np.sqrt((rppg_lf_hf - gt_lf_hf) ** 2)

        ## Add the metrics to the table
        table.add_row([subject, 
                       mae_pr, rmse_pr,
                       mae_meanNN, rmse_meanNN,
                       mae_sdnn, rmse_sdnn,
                       mae_rmssd, rmse_rmssd,
                       mae_pnn50, rmse_pnn50,
                       mae_lf, rmse_lf,
                       mae_hf, rmse_hf,
                       mae_lf_hf, rmse_lf_hf])

## Print the table
print(table)



+---------+---------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------+----------------------+----------------------+-----------------------+-----------------------+
| Subject |        MAE_PR       |       RMSE_PR       |     MAE_MeanNN     |    RMSE_MeanNN     |      MAE_SDNN      |     RMSE_SDNN      |     MAE_RMSSD      |     RMSE_RMSSD     |     MAE_pNN50      |     RMSE_pNN50     |         MAE_LF        |        RMSE_LF        |        MAE_HF        |       RMSE_HF        |    MAE_LF_HF_Ratio    |    RMSE_LF_HF_Ratio   |
+---------+---------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+---------------------

---

## Notes: Task 2 and Task 3 Later